In [1]:
import tensorflow as tf
import os 
import numpy as np
import pandas as pd
from model import CNN_Model,Params

C:\Users\Armughan.Shahid\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Policy_Network(CNN_Model):
    def __init__(self,param_dict={},restore_params=False,pickle_file_path=""):
        CNN_Model.__init__(self,param_dict,restore_params,pickle_file_path)
        
    def form_loss(self,logits,targets):
        entropies=self.params.loss_fn(labels=targets,logits=logits)
        return entropies
        
    def Build_model(self):
        self.build_model_till_logits()
        with tf.variable_scope(self.params.name_scope):
            
#             self.Advantage=self.form_placeholder((None,1),tf.float32)
            self.probs_all_actions=tf.nn.softmax(self.logits)#converting to probs
            log_likelihood_all_actions=tf.log(self.probs_all_actions)
        
            #which action to take
            self.desired_action=tf.multinomial(log_likelihood_all_actions,num_samples=1)

#           we will be using sparse softmax cross entropy function which will give us neg ;p likelihoods for yje selected action
            self.neg_log_likelihood=self.form_loss(self.logits,self.desired_action[:][0])
           #computing gradients 
            optimizer=self.params.optimizer_fn(learning_rate=self.lr_placeholder)
            self.grads_and_vars=optimizer.compute_gradients(loss=self.neg_log_likelihood,var_list=self.model_trainable_variables)
            
            
#             #for easy manipulation of grads
            self.grads=np.array([grad for grad,var_name in self.grads_and_vars])
            
#             #placeholder for all gradients (list of placeholder and var name tuples)
#             print (self.grads_and_vars[0])
            self.grad_placeholders=[self.form_placeholder(grad.get_shape(),tf.float32) for (grad,_) in self.grads_and_vars]
            self.grads_and_vars_feed=[(self.grad_placeholders[i],self.grads_and_vars[i][1]) for i in np.arange(len(self.grads_and_vars)) ]
            #linking placeholder and var name
            
#             #taking grad step based on the values of grads received 
            self.train_op=optimizer.apply_gradients(grads_and_vars=self.grads_and_vars_feed,global_step=self.step_no)
            
            self.initializer=tf.global_variables_initializer()

In [3]:
params={
    'input_shape':[None, 35, 190, 1],
    'num_outputs':3,
    
    'layer_hierarchy':[
        {'layer_type':'conv_layer','kernel_size':8,'kernel_strides':1,'num_filters':16,'padding':'valid'},
        {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'conv_layer','kernel_size':4,'kernel_strides':1,'num_filters':32,'padding':'valid'},
        {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'flattening_layer'},
        {'layer_type':'fc_layer','num_hidden_units':256},
        {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'dropout_layer','dropout_probability':0.5},
        {'layer_type':'fc_layer','num_hidden_units':100},
        {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'dropout_layer','dropout_probability':0.5}
        
    ],
    'initializer_fn':tf.contrib.layers.variance_scaling_initializer,
    'activation_fn':tf.nn.relu,
    'loss_fn':tf.nn.sparse_softmax_cross_entropy_with_logits, #carefull
    'learning_rate':0.001,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/',
    'name_scope':'neural_network_bn'
}

In [4]:
tf.reset_default_graph()
model=Policy_Network(params)
model.Build_model()

(<tf.Tensor 'neural_network_bn_1/gradients/neural_network_bn/conv2d/Conv2D_grad/tuple/control_dependency_1:0' shape=(8, 8, 1, 16) dtype=float32>, <tf.Variable 'neural_network_bn/conv2d/kernel:0' shape=(8, 8, 1, 16) dtype=float32_ref>)
